In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import pandas as pd
# import scipy as needed

In [4]:
#Import  and Election data

solar_raw = pd.read_csv("./deepsolar_tract.csv", encoding = "ISO-8859-1")
county_elec_raw = pd.read_csv("./county_2018_elections.csv", encoding = "ISO-8859-1")

In [21]:
print(solar_raw.iloc[:,8:].columns)

Index(['education_college', 'education_doctoral',
       'education_high_school_graduate', 'education_less_than_high_school',
       'education_master', 'education_population',
       'education_professional_school', 'employed', 'gini_index',
       'heating_fuel_coal_coke',
       ...
       'incentive_count_nonresidential', 'incentive_residential_state_level',
       'incentive_nonresidential_state_level', 'net_metering', 'feedin_tariff',
       'cooperate_tax', 'property_tax', 'sales_tax', 'rebate',
       'avg_electricity_retail_rate'],
      dtype='object', length=161)


In [25]:
solar_raw[['median_household_income', 'education_high_school_graduate', 'education_bachelor', 'population', 'employed', 'gini_index', 'education_population']]

,median_household_income,education_high_school_graduate,education_bachelor,population,employed,gini_index,education_population
0,64232.0,1757,569,6577,3569,0.3490,4548
1,49778.0,767,674,8020,4795,0.4074,3533
2,63355.0,1541,854,6971,3550,0.3926,4480
3,71599.0,1095,640,6328,3483,0.3949,3456
4,68401.0,982,654,5264,2887,0.4463,3348
...,...,...,...,...,...,...,...
72532,70000.0,1099,1612,6629,3964,0.4422,4751
72533,39508.0,803,95,2958,1178,0.3818,1749
72534,50286.0,1343,426,4889,2291,0.3857,3314
72535,52750.0,571,134,1816,658,0.3852,1337


In [62]:
solar_grouped_temp = solar_raw.groupby(['county', 'state'], as_index=False)
solar_grouped_sumed = solar_grouped_temp.sum().sort_values(['county', 'state'])
solar_grouped_avged = solar_grouped_temp.mean().sort_values(['county', 'state'])
ratio_panels_to_housing_units = solar_grouped_sumed['tile_count'] / solar_grouped_sumed['housing_unit_count']
solar_grouped_sums = solar_grouped_sumed[['county', 'state', 'tile_count', 'solar_system_count', 'total_panel_area', 'land_area', 'total_area', 'population', 'education_high_school_graduate', 'education_bachelor', 'employed']].sort_values(['county', 'state'])
solar_grouped_avgs = solar_grouped_avged[['county', 'state', 'population_density', 'average_household_income', 'race_asian_rate', 'race_black_africa_rate', 'race_indian_alaska_rate', 'race_islander_rate', 'race_white_rate', 'race_other_rate', 'race_two_more_rate', 'diversity','voting_2016_dem_percentage', 'voting_2016_gop_percentage', 'avg_electricity_retail_rate', 'gini_index']].sort_values(['county', 'state'])

solar_grouped_all = pd.merge(solar_grouped_sums, solar_grouped_avgs, on=['county', 'state']).sort_values(['county', 'state'])
solar_grouped_all['ratio_panels_to_housing_units'] = ratio_panels_to_housing_units
solar_grouped_all['education_highschool_percentage'] = solar_grouped_all['education_high_school_graduate'] / solar_grouped_all['population']
solar_grouped_all['education_bachelor_percentage'] = solar_grouped_all['education_bachelor'] / solar_grouped_all['population']
solar_grouped_all['employed_percentage'] = solar_grouped_all['employed'] / solar_grouped_all['population']
solar_grouped_all = solar_grouped_all.drop(columns=['education_high_school_graduate', 'education_bachelor', 'employed'])
solar_grouped_all = solar_grouped_all.sort_values(['state', 'county'])

solar_grouped_all = solar_grouped_all.drop([2090])

solar_grouped_all.head()

,county,state,tile_count,solar_system_count,total_panel_area,land_area,total_area,population,population_density,average_household_income,...,race_two_more_rate,diversity,voting_2016_dem_percentage,voting_2016_gop_percentage,avg_electricity_retail_rate,gini_index,ratio_panels_to_housing_units,education_highschool_percentage,education_bachelor_percentage,employed_percentage
107,Autauga County,al,28.0,23.0,2567.542490,594.436015,604.388097,55221,810.895779,64064.299480,...,0.017455,0.342924,0.239569,0.734358,9.27,0.424858,0.001240,0.220604,0.088680,0.434364
116,Baldwin County,al,254.0,201.0,18351.453746,1589.783989,2027.311190,195121,430.466918,66657.723781,...,0.016964,0.213399,0.195653,0.773515,9.27,0.439061,0.002387,0.202085,0.135239,0.440511
128,Barbour County,al,8.0,6.0,1010.677426,884.876361,904.515217,26932,133.333457,44485.459066,...,0.011599,0.527576,0.466603,0.522714,9.27,0.457533,0.000677,0.240977,0.052428,0.319211
197,Bibb County,al,2.0,1.0,225.353902,622.582300,626.168790,22604,46.520710,52656.067494,...,0.017350,0.285977,0.214220,0.769662,9.27,0.428750,0.000223,0.290833,0.041851,0.366926
217,Blount County,al,18.0,6.0,960.822726,644.775890,650.628210,57710,93.144891,53897.009150,...,0.016832,0.090340,0.084699,0.898519,9.27,0.397744,0.000754,0.228366,0.058846,0.384491


In [119]:
solar_grouped_all.to_csv(r'./deepsolar_counties_cleaned.csv')

## Linear Regression
Linear regression to predict ratio of panels to housing units; using coefficient to determine what contributes most to the predictions.

In [111]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model as lm

linear_regression_data = solar_grouped_all.drop(columns=['race_indian_alaska_rate', 'race_islander_rate', 'race_two_more_rate', 'race_other_rate', 'voting_2016_dem_percentage'])

train, test = train_test_split(linear_regression_data, test_size=0.2)
train_y = train['ratio_panels_to_housing_units']
test_y = test['ratio_panels_to_housing_units']
train_x = train.iloc[:, 8:].drop(columns='ratio_panels_to_housing_units')
test_x = test.iloc[:, 8:].drop(columns='ratio_panels_to_housing_units')

In [112]:
model = lm.LinearRegression(fit_intercept=False)
model.fit(train_x, train_y)

fitted_y = model.predict(train_x)
predicted_y = model.predict(test_x)

In [113]:
def rmse(actual, predicted):
    return np.sqrt(np.mean((actual-predicted)**2))

In [114]:
training_error = rmse(train_y, fitted_y)
val_error = rmse(test_y, predicted_y)
(training_error, val_error)

(0.011033408566471847, 0.011667598413791029)

Our error is consistent across the training and testing data, so our model has no significant bias and any error is due to random chance.

In [118]:
d = {'features':train_x.columns, 'coefficients':model.coef_}
coefficients = pd.DataFrame(data=d)
coefficients

,features,coefficients
0,population_density,-3.430602e-07
1,average_household_income,1.375136e-07
2,race_asian_rate,1.480628e-01
3,race_black_africa_rate,-1.490562e-02
4,race_white_rate,-3.709938e-03
5,diversity,1.501945e-02
6,voting_2016_gop_percentage,-4.560570e-03
7,avg_electricity_retail_rate,2.001605e-03
8,gini_index,-8.988813e-03
9,education_highschool_percentage,-1.673925e-02
